In [9]:
import os, random, shutil, json

random.seed(67)  # reproducibility sixxx seveennnn

dataset_dir = "plushie-coco-dataset"
annotations_path = os.path.join(dataset_dir, "annotations", "result_coco.json")
src = os.path.join(dataset_dir, "images")
train_dst = os.path.join(dataset_dir, "train")
val_dst = os.path.join(dataset_dir, "val")

os.makedirs(train_dst, exist_ok=True)
os.makedirs(val_dst, exist_ok=True)

# --- load annotations ---
with open(annotations_path, "r") as f:
    coco = json.load(f)

# image list taken from JSON (safer than listing folder)
images_info = coco["images"]
all_filenames = [img["file_name"] for img in images_info]

# shuffle split
random.shuffle(all_filenames)
split_index = int(0.8 * len(all_filenames))
train_imgs = all_filenames[:split_index]
val_imgs   = all_filenames[split_index:]

# --- move files and update json paths ---
for img in images_info:
    fname = img["file_name"]

    if fname in train_imgs:
        new_rel = f"train/{fname}"
        shutil.move(os.path.join(src, fname), os.path.join(train_dst, fname))
    else:
        new_rel = f"val/{fname}"
        shutil.move(os.path.join(src, fname), os.path.join(val_dst, fname))

    # Update JSON fields:
    # img["file_name"] = fname                    # keep raw name
    img["path"] = new_rel                       # new relative path

print(f"Moved {len(train_imgs)} to train, {len(val_imgs)} to val.")

# --- save updated annotations ---
with open(annotations_path, "w") as f:
    json.dump(coco, f, indent=2)

print("✅ Updated paths written to results_coco.json")

Moved 48 to train, 13 to val.
✅ Updated paths written to results_coco.json


In [11]:
len(os.listdir(train_dst))/(len(os.listdir(train_dst))+ len(os.listdir(val_dst)))

0.7868852459016393